## Import Tools

In [12]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import time
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

## Dataset Processing

### Read in the original dataset

In [13]:
train_dl_origin = torch.load('Dataset/train_dl.pt', weights_only=False)
valid_dl_origin = torch.load('Dataset/valid_dl.pt', weights_only=False)

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,0:2]

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,0:2]

### CSI Processing: Take Modulus of complex matrices

In [14]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [15]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)
print(train_label.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
torch.Size([15000, 2])


### CSI Processing: Normalize to [0,1]

In [ ]:
# *Avoiding data leakage
train_min = train_CSI_modulus.min()
train_max = train_CSI_modulus.max()

train_CSI_norm = (train_CSI_modulus - train_min) / (train_max - train_min)
valid_CSI_norm = (valid_CSI_modulus - train_min) / (train_max - train_min)

print(train_CSI_norm.shape)
print(valid_CSI_norm.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


### Preparing Data for ML

In [ ]:
# Flattening data to be 2D, 1d vector
train_CSI_ML = train_CSI_norm.reshape(train_CSI_norm.shape[0], -1)
valid_CSI_ML = valid_CSI_norm.reshape(valid_CSI_norm.shape[0], -1)

print(train_CSI_ML.shape)

## Machine Learning: Linear Regression

In [ ]:

linreg = LinearRegression()

start_time = time.time()
linreg.fit(train_CSI_ML, train_label)
train_time = time.time() - start_time

# Evaluating on training set
train_pred = linreg.predict(train_CSI_ML)
train_mse = mean_squared_error(train_label, train_pred)
train_rmse = math.sqrt(train_mse)

# Evaluating on validation set
start_valid_time = time.time()
valid_pred = linreg.predict(valid_CSI_ML)
valid_time = time.time() - start_valid_time

valid_mse = mean_squared_error(valid_label, valid_pred)
valid_rmse = math.sqrt(valid_mse)

print('Training MSE: ', train_mse)
print('Valid MSE: ', valid_mse)
print('Train RMSE: ', train_rmse)
print('Valid RMSE: ', valid_rmse)
print('Training time taken: {:.2f}s'.format(train_time))
print('Validation time taken: {:.2f}s'.format(valid_time))

Training MSE:  22.084903717041016
Valid MSE:  22.998790740966797
Train RMSE:  4.6994578109651135
Valid RMSE:  4.795705447686169
Training time taken: 28.73s
Validation time taken: 0.04s


### Prepare Data for CNN

In [ ]:
# 4D shape for CNN
train_CSI_CNN = train_CSI_norm
valid_CSI_CNN = valid_CSI_norm

print('CNN shape:', train_CSI_CNN.shape)

## Neural Network: CNN Regressor

In [ ]:
class CNNRegressor(nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 1 * 408, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # Convolutional block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Convolutional block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Flattening
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

model = CNNRegressor()

# Count params
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters: {total_params:,}')

Total parameters: 6,737,090


In [ ]:
# Setup training
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_dataset = TensorDataset(train_CSI_CNN, train_label)
valid_dataset = TensorDataset(valid_CSI_CNN, valid_label)

# Dataloaders
BS = 32
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BS, shuffle=False)

# Training loop
num_epochs = 10
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

train_time = time.time() - start_time
print(f'Training completed in {train_time:.2f} seconds')

Epoch [1/10], Loss: 59.7920
Epoch [2/10], Loss: 35.9650
Epoch [3/10], Loss: 26.3386
Epoch [4/10], Loss: 21.3001
Epoch [5/10], Loss: 18.6085
Epoch [6/10], Loss: 17.3501
Epoch [7/10], Loss: 15.3680
Epoch [8/10], Loss: 13.9490
Epoch [9/10], Loss: 13.9110
Epoch [10/10], Loss: 12.4909
Training completed in 578.51 seconds


In [ ]:
# Eval
model.eval()
start_time = time.time()

valid_predictions = []
valid_labels_list = []

with torch.no_grad():
    for inputs, labels in valid_loader:
        outputs = model(inputs)
        valid_predictions.extend(outputs.tolist())
        valid_labels_list.extend(labels.tolist())

test_time = time.time() - start_time

# Calculate metrics and performance
valid_predictions = np.array(valid_predictions)
valid_labels_list = np.array(valid_labels_list)

mse = mean_squared_error(valid_labels_list, valid_predictions)
rmse = math.sqrt(mse)

print(f'Validation MSE: {mse:.4f}')
print(f'Validation RMSE: {rmse:.4f}')
print(f'Testing time: {test_time:.4f}s')

Validation MSE: 14.2634
Validation RMSE: 3.7767
Testing time: 9.8696s
